# Coursework 2: Cryptography

[X] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 2.1 ElGamal Public Key and Digital Signature (50%)

### Please run the following helper functions first

The function **string_to_ints** can **convert a string into a Unicode vector**. For example, when we input the string Message = "Hi"， then the function will change it into the Unicode vector [72,105].
  
Then the function **ints_to_string** can **convert a Unicode vector into a string** e.g. [72,105] ==> "Hi".

This is a very powerful conversion, and even **emoji can be transformed and turned back**. e.g. "🙂" ==> [128578] ==> "🙂"

Function **isprime** checks if the input is a prime.

Function **primitive** finds the primitive element of a prime number.

In [1]:
# Helper functions for messages which are strings

# Funtion for to convering string to vector of Int64
function string_to_ints(message::String)::Vector{Int64}
    return [ Int64(char) for char in message ]
end

# Funtion is used to convering vector to string by converting
# every element in array to a unicode char
function ints_to_string(int_message::Vector{Int64})::String
    return join(Char(int) for int in int_message)
end

# Function that checks if the input is prime
function isprime(x::Int64)::Bool
    if x < 3
        return x > 1
    elseif x % 2 == 0 || x % 3 == 0
        return false
    else
        for i in 5:(sqrt(x)):6
            if x % i == 0 || x % (i+2) == 0
                return false
                end
        return true
        end
    end
end

# Function to find a primitive element
function find_primitive(p::Int64)::Int64 # This function is used to find the first primitive element of p
    @assert isprime(p)
    primitive_element = 0
    for i in ceil(sqrt(p)) : p # only find primitive element bigger than √p
        result = 1
        judgement = 0
        for n in 1:p-1
            result = mod(result*i,p)
            if result == 1 # If x^n is equal to 1, then count once
                judgement += 1
            end 
        end
        if judgement == 1 # If x^n only occurs one time, x is a primitive element
            primitive_element = i
            return convert(Int64, primitive_element)
        end
    end
end


find_primitive (generic function with 1 method)

In [2]:
# examples:

@show string_to_ints("Hi 🙂")

@show ints_to_string([72, 105, 32, 128578])

@show find_primitive(11)

string_to_ints("Hi 🙂") = [72, 105, 32, 128578]
ints_to_string([72, 105, 32, 128578]) = "Hi 🙂"


find_primitive(11) = 6


6

### 2.1.1 Key Generation

From [this list](https://primes.utm.edu/lists/small/10000.txt) choose a prime number $p$ that is larger than 10,000. Find a primitive element $b$ in $\mathbb{F}_p$ such that $b > \sqrt{p}$. Choose an integer $a^{\prime} > \sqrt{p}$ as your private key and use it to generate $a = b^{a^{\prime}} \in \mathbb{F}_p^*$. 

$(p,b,a)$ is the public information. 

### Theory: The ElGamal Cryptography
1. The transmitted message is ($b^t,ma^t$), where b is a primitive element of prime p, a' is the private key and $a= b^{a'}$ is the public key.  

2. The message received can be recovered computing $ma^t(b^t)^{-a'} = ma^tb^{-a't} = ma^ta^{-t}$ mod $p$.

### Solution: Function to generate the private and public keys

Input p should be a prime number.

Outputs:
1. b is a primitive element of p (because the calculation of the primitive element of large prime numbers is very slow, in order to save time, we choose the first primitive element bigger than  $\sqrt{p}$).

2. a' is a random integer bigger than $\sqrt{p}$. We choose a number bigger than $\sqrt{p}$ and smaller than p, which is used as a private key.

3. a equals to $b^{a'}\in \mathbb{F}_p^*$, which is used as public key.

4. t is another random integer bigger than $\sqrt{p}$.  We choose a number bigger than $\sqrt{p}$ and smaller than p, which is used as a second private key.  

In [3]:
function findkey(p::Int64)::NTuple{4, Int64}
    b = find_primitive(p) # choose the first primitive element of p
    a′ = convert(Int,rand((ceil(sqrt(p)):p))) # randomly choose a' form √p to p
    a = powermod(b,a′,p)  # a = b^a'
    t = convert(Int,rand((ceil(sqrt(p)):p))) # randomly choose one t form √p to p
    return (b,a′,a,t)
end

findkey (generic function with 1 method)

### 2.1.2 Encryption

Design and implement a function `ElGamal_Encrypt`. Write necessary documentation. 

### Solution: Function to encrypt the Message

1. The inputs t,b,p,a can be generated with the findkey function.   

2. The other input ***message* is a string or integer(Smaller than p)** (Example for use of integer: **4-digit password will be transmitted faster**), e.g. message = "Hi" ; message = 4436

3. The function will return the encrypted string.

In [4]:
# function to encrypt the message accepts message as a string or an integer (Int type)
# (we could not type enforce this due to Julia's lack of signed-unsigned implicit conversion on function parameters)
function ElGamal_Encrypt(t::Int64, b::Int64, p::Int64, message::Union{Int64, String}, a::Int64)::Tuple{Int64,Union{Int64, Vector{Int64}}}
    b_t = powermod(b, t, p) # b^t mod p
    if message isa Int64
        @assert p > abs(message) # raise an Error when message is bigger than p
        if message >= 0
            return (b_t, mod(message * powermod(a, t, p), p))
        else
            negative_int = string('-',abs(message)) # If the message is a negative integer, change it to string
            return (b_t, [ mod(m * powermod(a, t, p), p) for m in string_to_ints(negative_int) ])
        end
    else  # If the message is a string, convert it to a tuple
        return (b_t, [ mod(m * powermod(a, t, p), p) for m in string_to_ints(message) ])
    end
end

ElGamal_Encrypt (generic function with 1 method)

### 2.1.3 Decryption

Design and implement a function `ElGamal_Decrypt`. Write necessary documentation. 

Use at least 3 examples to demonstrate the correctness of your encryption and decryption functions. 

### Solution: Function to decrypt the Message

This function can **convert the encrypted message back**. The inputs are the private key a',the prime number p and the message received.

In [5]:
# function to decrypt encrypted message accepts either a single integer (if original message was an integer)
# or a string (if the original message had been a string)
function ElGamal_Decrypt(a′::Int64, p::Int64, message::Tuple{Int64, Union{Int64, Vector{Int64}}})::Union{Int64, String}
    b_t, encrypted_message = message
    if encrypted_message isa Int64
        @assert p > abs(encrypted_message) # raise an Error when message is bigger than p
        return mod(encrypted_message * powermod(invmod(b_t, p), a′, p), p)
    else
        return ints_to_string([ mod(m * powermod(b_t,-a′,p),p) for m in encrypted_message ]) #change the Unicode vector into a string
    end
end

ElGamal_Decrypt (generic function with 1 method)

### Testing 

1.A prime number should be big enough to be set send an emoji (ex:161771, 187139, 619331, 710641) and a number smaller than 2,500,000, is necessary to not cause data overflow.   

2.For text messages, a prime number p larger than 10,000 is sufficient.    

3.The input should be **String** or **Integer(Smaller than p)**.

4.**If there is an error in the result, please change a bigger p**.


In [29]:
p = 104729 # p can be small for text
Message = 4436

(b,a′,a,t) = findkey(p)
Encrypted_message = ElGamal_Encrypt(t,b,p,Message,a)
println("The received message is: " ,ElGamal_Decrypt(a′,p,Encrypted_message))

The received message is: 4436


In [30]:
p = 104729 # p can be small for text
Message = -4436

(b,a′,a,t) = findkey(p)
Encrypted_message = ElGamal_Encrypt(t,b,p,Message,a)
println("The received message is: " ,ElGamal_Decrypt(a′,p,Encrypted_message))

The received message is: -4436


In [8]:
p = 11863 # p can be small for text
Message = "Imperial Coding Theory"  

(b,a′,a,t) = findkey(p)
Encrypted_message = ElGamal_Encrypt(t,b,p,Message,a)
println("The received message is: " ,ElGamal_Decrypt(a′,p,Encrypted_message))

The received message is: Imperial Coding Theory


In [9]:
p = 161771    # p should be bigger enough for emoji
Message = "ICanSendEmoji🙂!" 

(b,a′,a,t) = findkey(p)
Encrypted_message = ElGamal_Encrypt(t,b,p,Message,a)
println("The received message is: " ,ElGamal_Decrypt(a′,p,Encrypted_message))

The received message is: ICanSendEmoji🙂!


In [10]:
p = 161771 # big p also works for text
Message = "JuliaCoding"

(b,a′,a,t) = findkey(p)
Encrypted_message = ElGamal_Encrypt(t,b,p,Message,a)
println("The received message is: " ,ElGamal_Decrypt(a′,p,Encrypted_message))

The received message is: JuliaCoding


### 2.1.4 Digital Signature Sign

Design and implement a function `ElGamal_Sign` to digitally sign a Message. Write necessary documentation. 

### Theory: Digital Signature with ElGamal Signature Scheme
The digital signature should be constructed based on the message. It should be: $i = b^t$, $j = u(|m|-a'|i|)$, where b is the primitive element, a' the private key and a the public key. t is chosen *s.t.* $1\leq t\leq p-1$ and $gcd(t,p-1) = 1$. u is chosen *s.t.* $mod(ut,p-1) = 1$. We should transmit (i,j).

The received signature is valid if $a^{|i|}i^j$ equals to $b^{|m|}$.

In [11]:
function Sign_gen(p::Int64)::Tuple{Int64, Int64} #function to generate t and u used for the digital signature
    candidates = [ i for i in 1:p-1 if gcd(i, p-1) == 1 ]
    t_sign = candidates[rand((1:length(candidates)))] # randomly choose one t
    u_sign = invmod(t_sign,p-1) # mod(t*u,p-1) = 1
    return (t_sign,u_sign)
end

# function to generate the digital signature for the Message
function ElGamal_Sign(b::Int64, keys::Tuple{Int64, Int64}, p::Int64, a′::Int64, message::Union{Int64, String})::Tuple{Int64, Union{Int64, Vector{Int64}}} 
    t_sign, u_sign = keys
    i = powermod(b,t_sign,p) # i = b^t
    if message isa Int64
        @assert p > abs(message) # raise an Error when message is bigger than p
        if message >= 0
            return (i, mod(u_sign*(abs(message%p) - a′ * abs(i%p)),p-1))
        else
            negative_int = string('-',abs(message)) # If the message is a negative integer, change it to string
            return (i, [ mod(u_sign*(abs(m%p) - a′ * abs(i%p)),p-1) for m in string_to_ints(negative_int) ])
        end
    else  # If the message is a string, convert it to a tuple
        return (i, [ mod(u_sign*(abs(m%p) - a′ * abs(i%p)),p-1) for m in string_to_ints(message) ])
    end
end

ElGamal_Sign (generic function with 1 method)

### 2.1.5 Digital Signature Verification

Design and implement a function `ElGamal_SignVerify` to verify the digital sigature. Write necessary documentation. 

Use at least 3 examples to demonstrate the correctness of your functions for digital signature and verification. 

In [12]:
# This function is to decrypt the digital signature, determine whether the digital signature is valid
# accepts an (enforced by assertion not parameter type-enforced) integer as input or string
function ElGamal_SignVerify(Sign::Tuple{Int64, Union{Int64, Vector{Int64}}} ,a::Int64,b::Int64, message::Union{Int64, String},p::Int64)::Bool
    i, j = Sign
    if message isa String
        @assert j isa Vector{Int64}
        int_message = string_to_ints(message) # Convert the original Message to Unicode
        Sign_m = [ mod(powermod(a, abs(i%p), p)*powermod(i, n, p) , p) for n in j ]
        b_m = [ powermod(b,abs(m%p),p) for m in int_message ]
    elseif message < 0
        negative_int = string('-',abs(message))
        @assert j isa Vector{Int64}
        int_message = string_to_ints(negative_int) # Convert the original Message to Unicode
        Sign_m = [ mod(powermod(a, abs(i%p), p)*powermod(i, n, p) , p) for n in j ]
        b_m = [ powermod(b,abs(m%p),p) for m in int_message ]
    else
        @assert j isa Int64
        @assert p> abs(message) # raise an Error when message is bigger than p
        Sign_m = mod(powermod(a, abs(i%p), p)*powermod(i, j, p) , p)
        b_m = powermod(b, abs(message%p), p)
    end
    if b_m == Sign_m # If they are same, the signature is valid
        println("The decrypted digital signature is: ",Sign_m)
        println("The decrypted signature should be: ",b_m)
        println("Valid")
        return true
    else
        println("The decrypted digital signature is: ",Sign_m)
        println("The decrypted signature should be: ",b_m)
        println("Not Valid")
        return false
    end
end

ElGamal_SignVerify (generic function with 1 method)

### Testing 

**Input for this part is the same as in the previous testing section**.

**If there is an error in the result, please change a bigger p**.

In [13]:
p = 11863 # p can be small for text
Message = 4436 # non-negative integer can run faster!

(b,a′,a,t) = findkey(p) # find the private and public keys
Keys_sign= Sign_gen(p) # find the t and u for digital signature
Sign = ElGamal_Sign(b,Keys_sign,p,a′,Message) # the digital signature
ElGamal_SignVerify(Sign,a,b,Message,p) # judgement

The decrypted digital signature is: 4088
The decrypted signature should be: 4088
Valid


true

In [14]:
p = 15467 # p can be small for text
Message = -4436

(b,a′,a,t) = findkey(p) # find the private and public keys
Keys_sign= Sign_gen(p) # find the t and u for digital signature
Sign = ElGamal_Sign(b,Keys_sign,p,a′,Message) # the digital signature
ElGamal_SignVerify(Sign,a,b,Message,p) # judgement

The decrypted digital signature is: [4958, 10885, 10885, 11842, 2993]
The decrypted signature should be: [4958, 10885, 10885, 11842, 2993]
Valid


true

In [15]:
p = 11863 # p can be small for text
Message = "Imperial Coding Theory" 

(b,a′,a,t) = findkey(p) # find the private and public keys
Keys_sign= Sign_gen(p) # find the t and u for digital signature

Sign = ElGamal_Sign(b,Keys_sign,p,a′,Message) # the digital signature
ElGamal_SignVerify(Sign,a,b,Message,p) # judgement

The decrypted digital signature is: [9545, 7533, 10311, 4586, 7653, 2989, 4774, 4096, 2184, 9621, 5101, 6681, 2989, 2550, 11370, 2184, 3811, 5578, 4586, 5101, 7653, 10731]
The decrypted signature should be: [9545, 7533, 10311, 4586, 7653, 2989, 4774, 4096, 2184, 9621, 5101, 6681, 2989, 2550, 11370, 2184, 3811, 5578, 4586, 5101, 7653, 10731]
Valid


true

In [16]:
p = 161771 # p should bigger enough for emoji
Message = "ICanSendEmoji🙂!"

(b,a′,a,t) = findkey(p) # find the private and public keys
Keys_sign= Sign_gen(p) # find the t and u for digital signature

Sign = ElGamal_Sign(b,Keys_sign,p,a′,Message) # the digital signature
ElGamal_SignVerify(Sign,a,b,Message,p) # judgement

The decrypted digital signature is: [87165, 23378, 13964, 84823, 160603, 114297, 84823, 18962, 68124, 41148, 65738, 160422, 108109, 141335, 148555]
The decrypted signature should be: [87165, 23378, 13964, 84823, 160603, 114297, 84823, 18962, 68124, 41148, 65738, 160422, 108109, 141335, 148555]
Valid


true

In [17]:
p = 161771 # Big prime also works for text
Message = "JuliaCoding"

(b,a′,a,t) = findkey(p) # find the private and public keys
Keys_sign= Sign_gen(p) # find the t and u for digital signature

Sign = ElGamal_Sign(b,Keys_sign,p,a′,Message) # the digital signature
ElGamal_SignVerify(Sign,a,b,Message,p) # judgement

The decrypted digital signature is: [48306, 30660, 107021, 108109, 13964, 23378, 65738, 18962, 108109, 84823, 152997]
The decrypted signature should be: [48306, 30660, 107021, 108109, 13964, 23378, 65738, 18962, 108109, 84823, 152997]
Valid


true

## 2.2 RSA Public Key and Digital Signature (50%)

### 2.2.1 Key Generation

From [this list](https://primes.utm.edu/lists/small/10000.txt) choose two prime numbers $p_1$ and $p_2$ that are larger than 10,000. Calculate $n=p_1 p_2$ and $t=(p_1-1)(p_2-1)$. Find an $e > 10,000$ such that $\text{gcd}(e,t)=1$. 

$(n,e)$ is the public key. 

### Solution:
The first stage in RSA Cryptography is the key generation stage. In this stage the following steps need to be followed:

1. Pick 2 distinct prime numbers. For testing we picked the following pairs : $ (17627, 18121), (12241, 983), (42169, 13127)$ . As a note, in reality, these numbers would have to be very large, however, this is a prototype and the functionality we used relatively smaller prime numbers to avoid potential overflowing and large running times.


2. Let $n=p1*p2$ and $t=(p1-1)(p2-1)$. For the prime numberswe have chosen the following n, t pairs would be generated:  $(319418867, 319383120), (12032903, 12019680), (553552463, 553497168) $


3. The public key is $(n,e)$ where e is a number between 1 and t with $gcd(e,t) = 1$.


In [18]:
#input: the prime numbers pair (p1,p2)
#output: the public key (n,e)

using Random

function public_keys(primes::Tuple{Int64, Int64})::Tuple{Int64, Int64}
    p1, p2 = primes
    e = 0
    while gcd(e, (p1-1)*(p2-1)) != 1
      e = rand(3:(p1-1)*(p2-1))  # randomly choose an e to check if it satisfies the condition gcd(e,t) = 1
    end
    return (p1*p2, e)
end

public_keys (generic function with 1 method)

In [19]:
#Testing
primes = (17627,18121) # Change p1 and p2 here, p1 = 17627 p2 =18121 
@show public_keys(primes)

primes = (12241, 200903)
@show public_keys(primes)

primes = (42169, 13127)
@show public_keys(primes)

public_keys(primes) = (319418867, 95305271)
public_keys(primes) = (2459253623, 1303589017)
public_keys(primes) = (553552463, 322710245)


(553552463, 322710245)

### 2.2.2 Encryption

Design and implement a function `RSA_Encrypt`. Write necessary documentation. 

### solution:

1.In the encryption stage the message m has to be transformed to $c = m^e$ mod $n$.  

2.The inputs public_key can be generated with the public_keys function.   

3.The other input ***message* is a string or integer(smaller than n)** (Example for use of integer: **4-digit password will be transmitted faster**), e.g. message = "Hi" ; message = 4436

4.The function will return the encrypted string.

In [20]:
# inputs: public key(n,e), message accepts an NON-NEGATIVE integer(smaller than n) or string 
# (we could not type enforce this due to Julia's lack of signed-unsigned implicit conversion on function parameters)
# output: encrypted message either a single integer (if message was a single integer) 
#.        or vector of integers (if message was strings)
function RSA_Encrypt(public_key::Tuple{Int64, Int64}, message::Union{Int64, String})::Union{Int64, Vector{Int64}}
    n, e = public_key
    if message isa String # If the message is a string, convert it to a tuple
        encrypted = [ powermod(m, e, n) for m in string_to_ints(message) ]
    elseif message < 0 # If the message is a negative integer, change it to string
        @assert n > abs(message)  # raise an Error when message is bigger than p
        negative_int = string('-',abs(message))
        encrypted = [ powermod(m, e, n) for m in string_to_ints(negative_int) ]
    else
        @assert n > abs(message)  # raise an Error when message is bigger than p
        encrypted = powermod(message, e, n)
    end
    return encrypted
end

RSA_Encrypt (generic function with 1 method)

In [21]:
#Testing (input message is a string or integer(Smaller than n))
primes = (17627,18121)
message = "ICanTransformEmoji🙂" 
@show RSA_Encrypt(public_keys(primes), message)

primes = (17627,18121)
message = 4436 # non-negative integer can run faster!
@show RSA_Encrypt(public_keys(primes), message)

primes = (17627,18121)
message = -4436 # Negative number can still correct throuth string
@show RSA_Encrypt(public_keys(primes), message)


RSA_Encrypt(public_keys(primes), message) = [300940292, 114705302, 139957317, 78360534, 9047108, 58080967, 139957317, 78360534, 268758596, 69538208, 152095227, 58080967, 240470369, 67215738, 240470369, 152095227, 7808255, 114749944, 73293133]


RSA_Encrypt(public_keys(primes), message) = 85551185
RSA_Encrypt(public_keys(primes), message) = [60526653, 106740441, 106740441, 12665327, 277500417]


5-element Vector{Int64}:
  60526653
 106740441
 106740441
  12665327
 277500417

### 2.2.3 Decryption

Design and implement a function `RSA_Decrypt`. Write necessary documentation. 

Use at least 3 examples to demonstrate the correctness of your encryption and decryption functions. 

The message can be recovered doing the following computation: $ c^d = m^{de} = m$ mod $n$.

$d$ is the private key where $d$ is between $1$ and $t$ and $d*e$ mod $t=1$.

In [22]:
#inputs: prime pair(p1,p2), public key(n,e), encrypted message
#        encrypted_message can be an single int (if the original message was an integer) 
#        or an array of integers (if the original message was a string)
#output: decoded message m either an integer (if original message was an int)
#.       or a string (if the original message was a string)
function RSA_Decrypt(prime_pair::Tuple{Int64, Int64}, public_key::Tuple{Int64, Int64}, encrypted_message::Union{Int64, Vector{Int64}})::Union{Int64, String}
    n, e = public_key
    p1, p2 = prime_pair
    private_key = invmod(e, (p1 - 1)*(p2 - 1))
    if encrypted_message isa Int64 # for non-negative message
        @assert n > abs(message)  # raise an Error when message is bigger than p
        return powermod(encrypted_message, private_key, n)
    else # for negative and string message
        return ints_to_string([ powermod(e, private_key, n) for e in encrypted_message ])
    end
end

RSA_Decrypt (generic function with 1 method)

**If there is an error in the result, please change a bigger p**.

In [23]:
###TEST CASES### The input message should be an integer(Smaller than n) or String
#If there is an error in the result, please change a bigger p.

message = 4436
primes = (17627, 18121)
public_key = public_keys(primes)
println("The sent message is:     ",message)
println("The received message is: " ,RSA_Decrypt(primes, public_key, RSA_Encrypt(public_key, message)))

message = -4436
primes = (17627, 18121)
public_key = public_keys(primes)
println("The sent message is:     ",message)
println("The received message is: " ,RSA_Decrypt(primes, public_key, RSA_Encrypt(public_key, message)))

message = "ImperialCodingTheory"
primes = (17627, 18121)
public_key = public_keys(primes)    
println("The sent message is:     ",message)
println("The received message is: " ,RSA_Decrypt(primes, public_key, RSA_Encrypt(public_key, message)))

message = "!!FourthYear!!"
primes = (12241, 983)
public_key = public_keys(primes)
println("The sent message is:     ",message)
println("The received message is: " ,RSA_Decrypt(primes, public_key, RSA_Encrypt(public_key, message)))

message = "ICanSendEmoji🙂"
primes = (42169, 13127)
public_key = public_keys(primes)
println("The sent message is:     ",message)
println("The received message is: " ,RSA_Decrypt(primes, public_key, RSA_Encrypt(public_key, message)))


The sent message is:     4436
The received message is: 4436
The sent message is:     -4436
The received message is: -4436
The sent message is:     ImperialCodingTheory
The received message is: ImperialCodingTheory
The sent message is:     !!FourthYear!!
The received message is: !!FourthYear!!
The sent message is:     ICanSendEmoji🙂
The received message is: ICanSendEmoji🙂


### 2.2.4 Digital Signature Sign

Design and implement a function `RSA_Sign` to digitally sign a Message. Write necessary documentation. 

The main steps for RSA signature are:

1. Pick a pair of 2 distinct prime numbers and compute $ n = p1* p2$ and $ t = (p1-1)* (p2-1)$.
2. (n, e) will be the public key where $ 1<e<t $ and $gcd(e,t)=1$
4. $d$ is the private key where $d$ is between $1$ and $t$ and $d*e$ mod $t=1$.

The signed message will be $y = m^d$ mod $n$ and the pair (m, y) will be sent where m is the original message.

In [24]:
#inputs: prime pair(p1, p2), public key (n,e), message m (either an positive integer or a string)
#outputs: signed message y
function RSA_Sign(prime_pair::Tuple{Int64, Int64}, public_key::Tuple{Int64, Int64}, message::Union{String, Int64})::Union{Int64, Vector{Int64}}
    n, e = public_key
    p1, p2 = prime_pair
    private_key = invmod(e, (p1 - 1)*(p2 - 1))
    if message isa Int64
        @assert n > abs(message)  # raise an Error when message is bigger than p
        if message >= 0
            return powermod(message, private_key, n)
        else
            negative_int = string('-',abs(message)) # If the message is a negative integer, change it to string
            return [ powermod(m, private_key, n) for m in string_to_ints(negative_int) ]
        end
    else # If the message is a string, convert it to a tuple
        return [ powermod(m, private_key, n) for m in string_to_ints(message) ]
    end
end

RSA_Sign (generic function with 1 method)

In [25]:
#Testing: Input same as above
message = "ImperialCodingTheory"
primes = (12241, 983)
public_key = public_keys(primes)
@show RSA_Sign(primes , public_key, message)

message = "ICanSendEmoji🙂"
primes = (42169, 13127)
public_key = public_keys(primes)
@show RSA_Sign(primes , public_key, message)

message = 4436
primes = (17627, 18121)
public_key = public_keys(primes)
@show RSA_Sign(primes , public_key, message)

message = -4436
primes = (17627, 18121)
public_key = public_keys(primes)
@show RSA_Sign(primes , public_key, message)


RSA_Sign(primes, public_key, message) = [9555602, 8364053, 894238, 3614842, 4821433, 3441357, 5241001, 10365496, 11315714, 7391298, 342141, 3441357, 7820368, 8094486, 4197794, 8144275, 3614842, 7391298, 4821433, 168988]
RSA_Sign(primes, public_key, message) = [448686311, 113787627, 75399912, 474294332, 110380047, 301616594, 474294332, 441940740, 456616552, 187673958, 365617523, 55698441, 360215450, 202620695]


RSA_Sign(primes, public_key, message) = 215890578
RSA_Sign(primes, public_key, message) = [117813071, 28811836, 28811836, 174904992, 137399469]


5-element Vector{Int64}:
 117813071
  28811836
  28811836
 174904992
 137399469

### 2.2.5 Digital Signature Verification

Design and implement a function `RSA_SignVerify` to verify the digital sigature. Write necessary documentation. 

Use at least 3 examples to demonstrate the correctness of your functions for digital signature and verification. 

To verify the signature the receiver has to check if $y^e=m$ mod $n$. If this is satisfied the signature is validated.

In [26]:
#input: public key
#.      signature that is either an Int or vector of Ints depending on if message was single integer or string
#.      message that is either an positive integer (enforced with assertion not parameter UInt type-checking or string
#output: boolean indicating whether verification passed or not
function RSA_SignVerify(public_key::Tuple{Int64, Int64}, signature::Union{Int64, Vector{Int64}}, message::Union{String, Int64})::Bool
    n, e = public_key
    if message isa Int64
        @assert n> abs(message)  # raise an Error when message is bigger than p
        if message >= 0
            return message == powermod(signature, e, n)
        else # for negative message
            negative_int = string('-',abs(message))
            return negative_int == ints_to_string([ powermod(s, e, n) for s in signature ])
        end
    else # for string message
        return message == ints_to_string([ powermod(s, e, n) for s in signature ])
    end
end

RSA_SignVerify (generic function with 1 method)

**If there is an error in the result, please change a bigger p**.

In [27]:
#Testing:  Input same as above
#If there is an error in the result, please change a bigger p.

message = 4436
primes = (17627, 18121)
public_key = public_keys(primes)
@show RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message)

message = -4436
primes = (17627, 18121)
public_key = public_keys(primes)
@show RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message)

message = "ImperialCodingTheory"
primes = (12241, 983)
public_key = public_keys(primes)
@show RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message)

message = "ICanSendEmoji🙂"
primes = (42169, 13127)
public_key = public_keys(primes)
@show RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message)

message = "JuliaCoding"
primes = (42169, 13127)
public_key = public_keys(primes)
@show RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message)

RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message) = true


RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message) = true
RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message) = true
RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message) = true
RSA_SignVerify(public_key, RSA_Sign(primes, public_key, message), message) = true


true

## Highlight

1. Both of the cryptography approaches were explained and the functions are supported with theory.

2. The code was deeply optimised and we made use of fast functions such as powermod and invmod. The transmitted message can be of type Int and also String: **various languages and symbols can be supported** including emoji. Int and Strings have customized handling for both types to skip redundant conversions/calculations if the message is an Int (Example a 4-digit password will be transmitted faster).

3. The code is modular and has clear comments and variable names that help the reader understand each step.

4. All of the functions were thoroughly tested with at least 3 examples.

5. All the private keys are randomly picked to enforce highly secure transmissions.

